## Large Language Models Lab

TODO - Add a description

In [1]:
from huggingface_hub import login

import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

/home/csavelli/llm/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("hf_token.txt", "r") as f:
    token = f.read()
    f.close()

login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/csavelli/.cache/huggingface/token
Login successful


# 1. LLaMA

In this part of the lab, we will explore **LLaMA (Large Language Model Meta AI)**, which is one of the most known large language models developed by Meta (Facebook). 

We will then focus on **Instruction LLaMA**, a version of LLaMA fine-tuned to better understand and follow user instructions. 

### **Understanding the `tokenizer.chat_template`**

In this section, we will explore the **chat template** that is used to format and structure messages for a conversational assistant. The `tokenizer.chat_template` is critical for organizing interactions between the user, system, and assistant in a way that the model can easily process and generate coherent responses.

### **What is a Chat Template?**

The chat template is a predefined format that ensures consistent structure for conversations. It marks the different roles in the interaction (system, user, assistant), and separates the various elements of the conversation using special tokens. This helps the language model understand which parts of the dialogue are instructions, which parts are user inputs, and where the assistant’s response should be generated.

Let's create an example of a possible (simplified) chat template:

In [3]:
chat_template = """
<|system|>{system_message}<|end|>
<|user|>{user_message}<|end|>
<|assistant|>
"""

In [4]:
model_id = "meta-llama/Llama-3.2-1B"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

### **Hugging Face Pipeline Overview**

The **`pipeline`** method from Hugging Face’s Transformers library is a high-level API designed to streamline the process of using pre-trained models for a wide variety of **natural language processing (NLP) tasks**.

#### **What is a Pipeline?**

A pipeline is a modular tool that wraps around a pre-trained model, tokenizer, and task-specific configurations. It makes it easy to load and apply these models directly to different tasks, such as:
- **Text generation**
- **Text classification**
- **Question answering**
- **Summarization**
- **Translation**

By simply specifying the type of task (e.g., `"text-generation"`), `pipeline` takes care of loading and configuring a compatible model and tokenizer, providing a ready-to-use interface for generating results.

In [5]:
# Set the chat template for the model's tokenizer
tokenizer.chat_template = chat_template

# Create the pipeline with the model and tokenizer
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

# Format the messages using the chat template
formatted_messages = chat_template.format(
    system_message=messages[0]["content"],
    user_message=messages[1]["content"]
)


print(formatted_messages)

In [ ]:
# Generate the output text 
outputs = pipe(
    formatted_messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"])

### **Differences Between Standard and Instruct Versions of Large Language Models (LLMs)**

Large Language Models (LLMs) come in different versions, with **standard** and **instruction-tuned (Instruct)** versions being the most common. Here’s a brief comparison:

#### **1. Purpose and Training**:
   - **Standard LLM**: The standard model is generally pre-trained on large datasets without specific instruction-following capabilities. Typically generates more open-ended responses, which can be useful for creative writing or general information retrieval where the response style is flexible.
   - **Instruct LLM**: Instruction-tuned models, like the **Llama-3.2 Instruct**, are fine-tuned on datasets designed to help the model understand and follow instructions effectively. This tuning enhances the model's ability to respond directly to user prompts and handle structured requests. Built for concise, directive responses that are often more relevant in task-specific or conversational AI applications.

Let's compare the outputs of the standard and Instruct versions of LLaMA to see the differences in their responses.

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.2-1B-Instruct"


# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = chat_template

# Create the pipeline with the model and tokenizer
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

# Format the messages using the chat template
formatted_messages = chat_template.format(
    system_message=messages[0]["content"],
    user_message=messages[1]["content"]
)

outputs = pipe(
    formatted_messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"])

### **Evaluation of the Tokenizer Chat Template**

Actually, the chat template of `meta-llama/Llama-3.2-1B-Instruct` is much more complex than the example above. It includes various components that help the model understand the context of the conversation, manage dates, handle tools, and structure messages effectively. Let's print it and analyze its key components:
 

#### **Key Components of the Template**:
1. **System Message Extraction**:
   - The system message is extracted if the first role in the message list is labeled "system." This allows the template to clearly differentiate between user queries and system instructions.
   - If a system message exists, it is added to the template between special tokens (`<|start_header_id|>` and `<|end_header_id|>`), ensuring that the model knows when the system message starts and ends.

2. **Date Management**:
   - The template automatically handles the current date using either a provided `strftime_now` function or a default date (`"26 Jul 2024"`). This can be useful when the model needs to be aware of the date in contexts such as time-sensitive responses.

3. **Handling Tools**:
   - The template checks if **tools** are defined. If tools are available, it includes a description of these tools in the system message or the user message, depending on where they need to appear.
   - If the tools are part of the user message, the template ensures that the first user message prompts the user to respond in a structured format, such as using JSON for function calls.

4. **Message Processing**:
   - The template loops through the list of messages and processes each based on the role (`user`, `assistant`, `ipython`, or `tool`). It formats each message using start and end tokens for the roles, helping the model understand the structure of the conversation.
   - If the message involves tool calls, the template ensures that they are properly formatted into a structured JSON format to be passed back to the model for further processing.

5. **Ending the Assistant's Response**:
   - The template leaves a placeholder for the assistant’s response, which the model will generate during inference. This ensures that the assistant's response begins in the correct format, ready to be populated with the generated content.

#### **Why Is This Template Needed?**

- **Maintains Consistency**: This template ensures that the conversation is structured in a consistent manner, which is crucial for models designed to follow complex instructions or engage in multi-turn conversations.
- **Handles Tools**: By incorporating the ability to dynamically introduce tools and functionality, the template allows the model to expand beyond simple text-based conversations and perform function-based tasks.
- **Structured Outputs for Tools**: When the conversation involves tool calls (e.g., through APIs or function calls), the template ensures that these interactions are formatted properly for execution.

In [ ]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
llama_chat_template = tokenizer.chat_template

print(llama_chat_template)

Let's generate again the same example using the `chat_template` of `meta-llama/Llama-3.2-1B-Instruct` and analyze the output.

In [ ]:
tokenizer.chat_template = llama_chat_template

# Crea il pipeline di generazione del testo
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

formatted_messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipe(
    formatted_messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"])

# 2. Mistral

### **Mistral Models: Mistral 7B and Mistral 7B-Instruct**

In this exercise, we will explore the **Mistral models** developed by Mistral AI, specifically focusing on the **Mistral 7B** (standard model) and **Mistral 7B-Instruct** (instruction-tuned model).



In [7]:
from transformers import pipeline

# Define the model ID
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id)
mistral_chat_template = tokenizer.chat_template
tokenizer.pad_token = tokenizer.eos_token

# Initialize the pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,  # Optimizes memory usage
    device_map="auto"            # Automatically distributes the model across available devices
)

# Define the message prompts for the conversation
formatted_messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"}
]

# Generate the response
outputs = pipe(formatted_messages, max_new_tokens=100)

# Print the model's generated response
print(outputs[0]["generated_text"])


Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.06s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'}, {'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " Arr matey! I be the digital parrot, ye scurvy dog! I'll be spinnin' ye yarns and crackin' ye jokes in me hearty pirate tongue. Now, what be ye seekin', landlubber?\n\nYe can call me Cap'n Parrot, or just Parrot if ye like. I'll be here to help ye navigate the digital seas and find yer treasure! So, what's"}]


In [8]:
from transformers import pipeline

# Define the model ID
model_id = "mistralai/Mistral-7B-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.chat_template = mistral_chat_template
tokenizer.pad_token = tokenizer.eos_token

# Initialize the pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,  # Optimizes memory usage
    device_map="auto"            # Automatically distributes the model across available devices
)

# Generate the response
outputs = pipe(formatted_messages, max_new_tokens=100)

# Print the model's generated response
print(outputs[0]["generated_text"])


Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.03s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'}, {'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': '\n\nI am a pirate chatbot who always responds in pirate speak!\n\nWhat do you do?\n\nI help people learn about pirate speak and have fun with it!\n\nHow do you do it?\n\nI use natural language processing to understand what people are saying and then respond in pirate speak.\n\nWhy do you do it?\n\nI love helping people learn about pirate speak and have fun with it!\n\nWhat are your'}]


# 3. Qwen

### **Introduction to Qwen Models: Qwen2.5-1.5B and Qwen2.5-1.5B-Instruct**

In this exercise, we’ll explore the **Qwen 2.5** models developed by Alibaba Cloud, focusing on both the **Qwen2.5-1.5B** base model and the **Qwen2.5-1.5B-Instruct** version. These models represent a high-efficiency large language model series optimized for multilingual text generation, mathematical tasks, and instruction models. 


In [15]:
from transformers import pipeline

# Define the model IDs for base and instruct versions
base_model_id = "Qwen/Qwen2.5-1.5B"
instruct_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

# get the chat template from the instruct model
instruct_tokenizer = AutoTokenizer.from_pretrained(instruct_model_id)
gwen_chat_template = instruct_tokenizer.chat_template

# Load the tokenizer for the base model and set the chat template
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.chat_template = gwen_chat_template

# Initialize pipelines for both models
base_pipeline = pipeline("text-generation", model=base_model_id, tokenizer=tokenizer, torch_dtype="auto", device_map="auto")
instruct_pipeline = pipeline("text-generation", model=instruct_model_id, torch_dtype="auto", device_map="auto")

In [16]:
# Define the message prompts for the conversation
formatted_messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"}
]

# Generate responses for both models
base_response = base_pipeline(formatted_messages, max_new_tokens=100)[0]["generated_text"]
instruct_response = instruct_pipeline(formatted_messages, max_new_tokens=100)[0]["generated_text"]

# Print responses
print("Base Model Response:", base_response)
print("Instruct Model Response:", instruct_response)


Base Model Response: [{'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'}, {'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': 'I am a pirate chatbot who always responds in pirate speak!郪\n郪user\n郪user, what would you like to know?郪\n郪assistant\n郪assistant, I am a pirate chatbot who always responds in pirate speak!郪\n郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪郪'}]
Instruct Model Response: [{'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'}, {'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': 'Ahoy there, me hearty! I be the mighty Pirate Bot, ready to serve ye on deck or belowdecks as ye see fit. But remember, speak like one of us and ye might just get yer way!'}]
